<a href="https://colab.research.google.com/github/thainazanfolin/great_expectations_tests/blob/main/great_expectations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install great_expectations

imports

In [2]:
!pip install pandas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import great_expectations as ge

## 1. Carregando o Dataset
Carregando o dataset a partir do arquivo raw no github.

In [13]:

url = 'https://raw.githubusercontent.com/thainazanfolin/great_expectations_tests/refs/heads/main/Sleep_Efficiency_table.csv'
df = pd.read_csv(url, delimiter=';')

# visão inicial do df
df.head()



,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,06/03/2021 01:00,06/03/2021 07:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,05/12/2021 02:00,05/12/2021 09:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,25/05/2021 21:30,25/05/2021 05:30,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,03/11/2021 02:30,03/11/2021 08:30,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,13/03/2021 01:00,13/03/2021 09:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


## 2. Análise exploratória dos dados
Iniciando a análise exploratória para entender possíveis testes úteis no momento do ETL.  <br><br>
Em outras palavras, fazendo a análise exploratória dos dados, conseguimos **entender o que nós esperamos deles, ou seja, quais são nossas expectativas.**

In [14]:
# número de linhas e colunas
print(f"\nLinhas e colunas: {df.shape[0]} linhas e {df.shape[1]} colunas")

# nomes das colunas
print("\nColunas:")
print(df.columns.tolist())

# tipos de dados e contagem (não nulos)
print("\nInfos:")
print(df.info())

# valores nulos por coluna
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# % de valores nulos
print("\n% de valores nulos por coluna:")
print((df.isnull().mean() * 100).round(2))

# Colunas numéricas
print("\nEstatísticas descritivas (colunas numéricas):")
print(df.describe())

# Colunas categóricas
print("\nEstatísticas descritivas (colunas categóricas):")
print(df.describe(include=['object']))

# Detecta possíveis colunas com valores duplicados
print("\nRegistros duplicados (linhas inteiras):", df.duplicated().sum())

# Verifica se há colunas que parecem ser identificadores únicos
print("\nColunas com valores únicos em todas as linhas (possíveis IDs):")
for col in df.columns:
    if df[col].is_unique:
        print(f"- {col}")



Linhas e colunas: 452 linhas e 15 colunas

Colunas:
['ID', 'Age', 'Gender', 'Bedtime', 'Wakeup time', 'Sleep duration', 'Sleep efficiency', 'REM sleep percentage', 'Deep sleep percentage', 'Light sleep percentage', 'Awakenings', 'Caffeine consumption', 'Alcohol consumption', 'Smoking status', 'Exercise frequency']

Infos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      452 non-null    int64  
 1   Age                     452 non-null    int64  
 2   Gender                  452 non-null    object 
 3   Bedtime                 452 non-null    object 
 4   Wakeup time             452 non-null    object 
 5   Sleep duration          452 non-null    float64
 6   Sleep efficiency        452 non-null    float64
 7   REM sleep percentage    452 non-null    int64  
 8   Deep sleep percentage   452 non-nul

# **Resultado da análise exploratória**
* 452 linhas e 15 colunas.

* Colunas: inclui variáveis sobre dados pessoais (ID, idade, gênero), hábitos de sono (hora de dormir, hora de acordar, duração do sono, eficiência do sono, percentagens de sono REM, profundo e leve, despertares), consumo de substâncias (cafeína, álcool, tabagismo) e exercícios físicos.

Ausência de Dados:

* Awakenings: 4.42% ausentes.

* Caffeine consumption: 5.53% ausentes.

* Alcohol consumption: 3.10% ausentes.

* Exercise frequency: 1.33% ausentes.


## **Usando o Great Expectations para ETL**

### O que vamos fazer com o great_expectations:

# Instalando Great_Expectations

In [17]:
import great_expectations as ge
from great_expectations.dataset import PandasDataset

# Converter o DataFrame pandas para PandasDataset (uma subclasse de Dataset)
class CustomPandasDataset(PandasDataset):
    pass


#convertendo o dataframe para GE
df_ge = CustomPandasDataset(df)




ModuleNotFoundError: No module named 'great_expectations.dataset'

# Definindo as expectativas

In [ ]:
# 'Age' : expectativa de estar entre 9 e 100
df_ge.expect_column_values_to_be_between('Age', 9, 100)


# 'Sleep duration': entre 4 e 12 horas
df_ge.expect_column_values_to_be_between('Sleep duration', 4, 12)

# 'Sleep efficiency': deve estar entre 0.50 e 1.00
df_ge.expect_column_values_to_be_in_set('Sleep efficiency', [x / 100 for x in range(50, 101)])

# Soma das porcentagens de sono (REM, Deep, Light) deve ser aproximadamente 100%
df_ge.expect_column_sum_to_be_between(
    column_list=["REM sleep percentage", "Deep sleep percentage", "Light sleep percentage"],
    min_value=99.9,  # não deve ser menor que 100%
    max_value=100.1  #  margem de erro
)

# 'Awakenings' entre 0 e 10
df_ge.expect_column_values_to_be_between('Awakenings', 0, 10)

#___________________________________________________________________________________________________________

# Verificar valores ausentes

# verificar se as colunas não possuem mais do que 5% de valores ausentes
df_ge.expect_column_pair_values_audit('Awakenings', None, max_value=0.05)
df_ge.expect_column_pair_values_audit('Caffeine consumption', None, max_value=0.05)
df_ge.expect_column_pair_values_audit('Alcohol consumption', None, max_value=0.05)
df_ge.expect_column_pair_values_audit('Exercise frequency', None, max_value=0.05)

# 'Gender' deve ter apenas 'Male' ou 'Female' como valores válidos
df_ge.expect_column_values_to_be_in_set('Gender', ['Male', 'Female'])

#  'Bedtime' e 'Wakeup time' deve seguir um formato específico (por exemplo, 'dd/mm/yyyy hh:mm')
df_ge.expect_column_value_lengths_to_be_in_range('Bedtime', min_value=16, max_value=16)
df_ge.expect_column_value_lengths_to_be_in_range('Wakeup time', min_value=16, max_value=16)

#  Verificar que 'Caffeine consumption', 'Alcohol consumption', 'Exercise frequency' são não-negativos
df_ge.expect_column_values_to_be_greater_than_or_equal_to('Caffeine consumption', 0)
df_ge.expect_column_values_to_be_greater_than_or_equal_to('Alcohol consumption', 0)
df_ge.expect_column_values_to_be_greater_than_or_equal_to('Exercise frequency', 0)

#  A soma de 'REM sleep percentage', 'Deep sleep percentage' e 'Light sleep percentage' deve ser 100
df_ge.expect_column_sum_to_be_between(
    column_list=["REM sleep percentage", "Deep sleep percentage", "Light sleep percentage"],
    min_value=99.9,
    max_value=100.1
)

# Verificar se 'Smoking status' tem valores válidos ('Yes' ou 'No')
df_ge.expect_column_values_to_be_in_set('Smoking status', ['Yes', 'No'])

# 'Age' deve ter valores numéricos válidos
df_ge.expect_column_values_to_be_of_type('Age', 'int64')

# Verificar que as porcentagens de sono (REM, Deep, Light) somem 100
df_ge.expect_column_sum_to_be_between(
    column_list=["REM sleep percentage", "Deep sleep percentage", "Light sleep percentage"],
    min_value=99.9,
    max_value=100.1
)

# Rodar as validações e mostrar o resultado
results = df_ge.validate()
print(results)


## Principais funções do GE

📊 **Expectativas de coluna<br>**
<br>expect_column_values_to_be_between<br>expect_column_values_to_be_in_set<br>expect_column_values_to_match_regex<br>expect_column_values_to_not_be_null<br>expect_column_mean_to_be_between<br>expect_column_median_to_be_between<br>expect_column_unique_value_count_to_be_between<br>expect_column_values_to_be_unique<br>expect_column_values_to_be_of_type<br>expect_column_values_to_be_in_type_list<br>expect_column_max_to_be_between<br>expect_column_min_to_be_between<br>expect_column_sum_to_be_between<br>expect_column_proportion_of_unique_values_to_be_between<br>expect_column_pair_values_to_be_equal<br>expect_column_values_to_be_increasing<br>expect_column_values_to_be_decreasing<br>expect_column_values_to_be_null<br>expect_column_values_to_match_strftime_format<br>expect_column_values_to_be_dateutil_parseable<br>expect_column_values_to_be_json_parseable<br>expect_column_values_to_match_like_pattern<br>expect_column_values_to_match_regex_list<br>expect_column_values_to_be_in_type_list<br>expect_column_values_to_be_of_type<br>expect_column_values_to_be_in_set<br>expect_column_values_to_not_be_in_set<br>expect_column_values_to_be_between<br>expect_column_values_to_be_null<br>expect_column_values_to_not_be_null<br>expect_column_values_to_be_unique<br>expect_column_values_to_not_match_regex<br>expect_column_values_to_not_match_regex_list<br>expect_column_values_to_match_regex<br>expect_column_values_to_match_regex_list<br>expect_column_values_to_be_in_type_list<br>expect_column_values_to_be_of_type<br>expect_column_values_to_be_in_set<br>expect_column_values_to_not_be_in_set<br>expect_column_values_to_be_between<br>expect_column_values_to_be_null<br>expect_column_values_to_not_be_null<br>expect_column_values_to_be_unique<br>expect_column_values_to_not_match_regex<br>expect_column_values_to_not_match_regex_list<br>expect_column_values_to_match_regex<br>expect_column_values_to_match_regex_list


<br><br>
**📋 Expectativas de Tabela<br>**<br>
<br>expect_table_row_count_to_be_between
<br>expect_table_column_count_to_be_between
<br>expect_table_columns_to_match_ordered_list
<br>expect_table_columns_to_match_set
<br>expect_table_row_count_to_equal

**🔗 Expectativas de Pares de Colunas**<br>
<br>expect_column_pair_values_to_be_equal
<br>expect_column_pair_values_A_to_be_greater_than_B
<br>expect_column_pair_values_A_to_be_less_than_B

🧪 Expectativas personalizadas
Você pode criar suas próprias expectativas personalizadas para atender a requisitos específicos do seu projeto.